In [2]:
import os
import cv2
import pickle
import datetime
import face_recognition

In [2]:
def generate_encodings(images_folder_path='./photos'):

    all_face_encodings = {}

    all_directories = os.listdir(images_folder_path)

    for folder in all_directories:
        
        face_encodings = []

        all_files = os.listdir(f'{images_folder_path}/{folder}')
        img_path = [filename for filename in all_files if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.gif', '.bmp'))] # Filter for image files (e.g., .jpg, .png, .jpeg)
        
        for img in img_path:
            print(f'{images_folder_path}/{folder}/{img}')
            image = face_recognition.api.load_image_file(f'{images_folder_path}/{folder}/{img}')
            encoding = face_recognition.api.face_encodings(image, model='large')
            # print(face_recognition.api.face_encodings(image, model='large'))
            if len(encoding) > 0:
                face_encodings.append(encoding[0])
            else:
                print('No face detected!')
                return
            
        all_face_encodings[folder] = face_encodings
        
    with open('./database/known_face_encodings', 'wb') as file:
        pickle.dump(all_face_encodings, file)

# generate_encodings("./photos")

In [2]:
def load_encondings(face_encodings_pkl_path: str) -> dict:

    with open(face_encodings_pkl_path, 'rb') as file:
        face_encodings = pickle.load(file)

    return face_encodings

load_encondings('./database/known_face_encodings')

{'23140736-BCU-ABHASH-RAI': [array([-0.13975768,  0.19197288,  0.08623125, -0.01944662, -0.02608149,
         -0.05216581, -0.10801405, -0.11858927,  0.16468582, -0.14436609,
          0.20888567, -0.03109686, -0.1625082 , -0.13827166, -0.0159538 ,
          0.14251807, -0.2225251 , -0.18928708, -0.03707772, -0.09682716,
          0.1402946 ,  0.03408594,  0.03444123,  0.00074216, -0.17644282,
         -0.38997096, -0.10423236, -0.09401594, -0.0119301 , -0.08564267,
          0.01870363, -0.00582178, -0.22153319, -0.07299553, -0.02506236,
          0.05219862, -0.00479974, -0.04467555,  0.14905073,  0.03577624,
         -0.17363094,  0.02868268,  0.05619542,  0.31705254,  0.11386447,
          0.06806765, -0.0090971 , -0.09048045,  0.08632563, -0.15958694,
          0.09410673,  0.17135787,  0.0707368 ,  0.05731313,  0.090533  ,
         -0.13491829,  0.06033682,  0.09290357, -0.20361947,  0.04986257,
          0.05936624, -0.05765337, -0.03416761, -0.00231101,  0.21852237,
          0

In [3]:
class Session:

    def __init__(self, face_location_model='hog', face_encoding_model = 'small'):

        self.face_location_model = face_location_model #'cnn' has better accuracy but uses GPU, 'hog' is faster with less accuracy uses cpu
        self.face_encoding_model = face_encoding_model #'large' model has better accuracy but is slower, 'small' model is faster

        self.__face_encodings_pkl_path = './database/known_face_encodings'
        with open(self.__face_encodings_pkl_path, 'rb') as file:
            self.__known_face_encodings = pickle.load(file)

    def get_current_time(self) -> str:
        '''Gets the current timestamp with seconds precision, converts it to string, and returns it'''
        return datetime.datetime.now().strftime('%H:%M:%S')

    def compare_faces(self, known: dict, unknown, tolerance=0.6):
        for student_identity, face_encoding in known.items():
            if True in face_recognition.api.compare_faces(face_encoding, unknown, tolerance=tolerance):
                return student_identity
        return None

    def start_session(self, camera_index=0, show_preview=True, scale_frame=0.25, desired_fps=2, tolerance=0.6):

        cap = cv2.VideoCapture(camera_index)
        frame_delay = int(1000 / desired_fps)  # Delay in milliseconds between frames based on the desired FPS
        
        while True:
            ret, frame = cap.read()

            small_frame = cv2.resize(frame, (0, 0), fx=scale_frame, fy=scale_frame) # Resize the frame for faster processing
            rgb_frame = small_frame[:, :, ::-1] # Convert the frame from BGR to RGB

            face_locations = face_recognition.face_locations(rgb_frame, model=self.face_location_model) # Find face locations and face encodings in the frame
            unknown_face_encodings = face_recognition.face_encodings(rgb_frame, face_locations) # Generate encodings of every faces in the frame in a list
            number_of_faces_detected = len(unknown_face_encodings)

            self.__identified_student_ids = []

            if number_of_faces_detected != 0: # Send data only if one or more person is detected
                time = self.get_current_time()
                print(f'[FACE DETECTION] {number_of_faces_detected} faces detected at {time}.')
                for unknown_face in unknown_face_encodings:
                    id = self.compare_faces(self.__known_face_encodings, unknown_face, tolerance=tolerance)
                    if id is None:
                        id = 'Unknown'
                    self.__identified_student_ids.append(id)

            if show_preview == True: 
                # Draw rectangles around detected faces and display names
                for (top, right, bottom, left), identity in zip(face_locations, self.__identified_student_ids):
                    top *= int(1 / scale_frame)
                    right *= int(1 / scale_frame)
                    bottom *= int(1 / scale_frame)
                    left *= int(1 / scale_frame)

                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                    cv2.putText(frame, identity, (left, bottom + 20), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 255, 0), 1)
                
                cv2.imshow('Face Detection', frame) # Display the frame with face rectangles
                
                if cv2.waitKey(frame_delay) & 0xFF == ord('q'): # Break the loop if 'q' key is pressed
                    cap.release() # Release the camera
                    cv2.destroyAllWindows() # Close the window
                    break

In [4]:
a = Session()
a.start_session()

[FACE DETECTION] 1 faces detected at 11:31:05.
[FACE DETECTION] 1 faces detected at 11:31:06.
[FACE DETECTION] 1 faces detected at 11:31:07.
[FACE DETECTION] 1 faces detected at 11:31:08.
[FACE DETECTION] 1 faces detected at 11:31:10.
[FACE DETECTION] 1 faces detected at 11:31:11.
[FACE DETECTION] 1 faces detected at 11:31:12.
[FACE DETECTION] 1 faces detected at 11:31:13.
[FACE DETECTION] 1 faces detected at 11:31:14.
[FACE DETECTION] 1 faces detected at 11:31:16.
[FACE DETECTION] 1 faces detected at 11:31:17.
[FACE DETECTION] 1 faces detected at 11:31:19.
[FACE DETECTION] 1 faces detected at 11:31:21.
[FACE DETECTION] 1 faces detected at 11:31:22.
[FACE DETECTION] 1 faces detected at 11:31:24.
